In [6]:
import pandas as pd
import numpy as np
import json
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics import accuracy_score#, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
import seaborn as sns
import matplotlib.pyplot as plt

## Dataset 1: Student Dropout Prediction

In [7]:
# Function 1: Data Preprocessing
def preprocess_data(path):
    """
    Function to preprocess the raw data.
    This can include tasks like handling missing values, encoding categorical variables, etc.
    """
    # Load the Data
    data = pd.read_csv(path, delimiter=';')

    # Data Preprocessing
    # Rename wrong column name
    data.rename(columns={'Nacionality': 'Nationality'}, inplace=True)
    data.rename(columns={'Daytime/evening attendance\t': 'Daytime/evening attendance'}, inplace=True)
    
    # Check for missing values
    #  print(data.isnull().sum()) # no null value
    return data

In [8]:
# Function 2: Feature Engineering
def engineer_features(data):
    """
    Function to engineer new features or transform existing ones.
    This can involve tasks like scaling, creating interaction terms, etc.
    """
    y = data['Target']  # Target variable
    X = data[['Previous qualification', "Mother's qualification", "Father's qualification"]]  # Features
#     X = data[['Admission grade', 'Inflation rate', 'Nationality', 'Daytime/evening attendance']]
    
    # Covert categorical target variable to numerical values
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    # X_reshaped = np.array(X).reshape(-1, 1)
    
    return X, y_encoded

In [9]:
# Function 3: Model Training
def train_model(X, y):
    """
    Function to train a decision tree model.
    """
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # Train decision tree classification model
    model = LogisticRegression(max_iter=5000)
    model.fit(X_train, y_train)

    # Predict on the test set
    y_pred = model.predict(X_test)

    return model, X_test, y_test, y_pred

In [10]:
# Function 5: Model Evaluation (for regression)
def evaluate_model(X_test, y_test, y_pred):
    """
    Function to evaluate a regression model.
    """
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)

    print("Mean Absolute Error:", mae)
    print("Mean Squared Error:", mse)
    print("R-squared:", r2)
    print("Accuracy:", accuracy)
    return accuracy

In [11]:
# Master Function to Control Workflow
def run_pipeline(path_to_data):
    """
    Master function to control the workflow/pipeline execution.
    """
    # Step 1: Data Preprocessing
    preprocessed_data = preprocess_data(path_to_data)
    
    # Step 2: Feature Engineering
    X, y = engineer_features(preprocessed_data)
    
    # Step 3: Model Training
    model, X_test, y_test, y_pred = train_model(X, y)
    
    # Step 4: Model Evaluation
    evaluation_result = evaluate_model(X_test, y_test, y_pred)
    
    return model, evaluation_result

In [12]:
path_to_data = "Data\data.csv"
model, accuracy = run_pipeline(path_to_data)

Mean Absolute Error: 0.8576807228915663
Mean Squared Error: 1.5308734939759037
R-squared: -0.9314491176636814
Accuracy: 0.4789156626506024


## Dataset 2: National Poll on Healthy Aging

In [8]:
# Load your dataset
data = pd.read_csv('Data/NPHA-doctor-visits.csv', delimiter=',')

# Selecting features (X) and target variable (y)
X = data[['Number of Doctors Visited']]
y = data['Phyiscal Health']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=3) # better score at random state 42

# Initialize the linear regression model
model = LinearRegression()

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test data
predictions = model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print('Mean Absolute Error:', mae)
print('Mean Squared Error:', mse)
print('R-squared:', r2)

threshold = 1  # Define your threshold here

# Calculate the percentage of predictions within the threshold
accuracy = np.mean(np.abs(predictions - y_test) <= threshold) * 100
print('Accuracy (within ±{} units): {:.2f}%'.format(threshold, accuracy))

Mean Absolute Error: 0.6316353331490185
Mean Squared Error: 0.654789442799183
R-squared: 0.05117109738582337
Accuracy (within ±1 units): 76.28%


## Dataset 3: Paper Reviews

In [9]:
# Load your dataset
path = r'E:\IBA\Semester 1\Machine Learning 1\Assignment\Assignment1\Data Piplines\Data\reviews.json'
with open(path, 'r', encoding='utf-8') as json_file:
    # Load JSON data into a Python dictionary
    data = dict(json.load(json_file))
    
flattened_data = []
for entry in data['paper']:
    for review in entry['review']:
        entry_data = {
            'paper_id': entry['id'],
            'preliminary_decision': entry['preliminary_decision'],
            'review_id': review['id'],
            'confidence': review['confidence'],
            'evaluation': review['evaluation'],
            'lan': review['lan'],
            'orientation': review['orientation'],
            'remarks': review['remarks'],
            'text': review['text'],
            'timespan': review['timespan']
        }
        flattened_data.append(entry_data)
data = pd.DataFrame(flattened_data)

# Encode string
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(data['preliminary_decision'])

# Selecting features (X) and target variable (y)
X = data[['confidence']]  # Feature: Confidence score
y = y_encoded  # Target: Preliminary decision (accept/reject)

y = np.array(y).reshape(-1, 1)

imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)
y_imputed = imputer.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_imputed, y_imputed, test_size=0.3, random_state=123)

# Initialize the Random Forest Classifier model
model = RandomForestClassifier()

# Train the model
model.fit(X_train, y_train.ravel())  # Note: ravel() is used to flatten y_train

# Make predictions on the test data
predictions = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
print('Accuracy:', accuracy)

Accuracy: 0.6147540983606558


## Rough work

In [ ]:
# data['Inflation rate']
# data.columns
# data.describe()
# plt.figure(figsize=(12, 8))
# sns.boxplot(data=data['Unemployment rate'])  # Exclude the target variable from the plot
# plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
# plt.title('Boxplot of the Data Features')
# plt.tight_layout()
# plt.show()
# print(model.predict([[2],[-0.3]]))
# print(data['preliminary_decision'].unique())